Json解析器

In [1]:
from typing import List

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

chat=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.7
)

c:\Users\86151\anaconda3\envs\langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3701: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
class Book(BaseModel):
    title: str = Field(description="书名")
    author: str =Field(description="作者")
    description: str = Field(description="书的简介")

In [3]:
query="请给我介绍学习中国历史的经典书籍"

parser=JsonOutputParser(pydantic_object=Book)

# 获取系统默认指令（通常是英文）
# format_instructions = parser.get_format_instructions()

# 手动定义中文的 JSON 结构要求
format_instructions = '''输出应格式化为符合以下 JSON 结构的 JSON 实例。
JSON结构
{
  "title": "书的标题",
  "author": "作者",
  "description": "书的简介"
}
'''

prompt=PromptTemplate(
    template="{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)

chain=prompt | chat | parser

chain.invoke({"query":query})

{'title': '中国历史',
 'author': '司马迁',
 'description': '《史记》是中国历史上第一部纪传体通史，记载了从黄帝到汉武帝时期的历史事件和人物，是一部非常重要的史学著作。'}

In [7]:
format_instructions=parser.get_format_instructions
format_instructions

<bound method JsonOutputParser.get_format_instructions of JsonOutputParser(pydantic_object=<class '__main__.Book'>)>

In [9]:
for s in chain.stream({"query":query}):
    print(s)

{}
{'title': ''}
{'title': '中国'}
{'title': '中国历史'}
{'title': '中国历史经典'}
{'title': '中国历史经典', 'author': ''}
{'title': '中国历史经典', 'author': '中国'}
{'title': '中国历史经典', 'author': '中国历史'}
{'title': '中国历史经典', 'author': '中国历史学家'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': ''}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代著名'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代著名的历史'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代著名的历史著作'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代著名的历史著作，'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代著名的历史著作，如'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代著名的历史著作，如《'}
{'title': '中国历史经典', 'author': '中国历史学家', 'description': '精选了中国古代著名的历史著作，如《史'}
{'title': '中国历史经典',

修复输出解析器

In [10]:
class Actor(BaseModel):
    name: str=Field(description="演员的姓名")
    film_names: List[str]=Field(description="演员参演的电影名称列表")

In [11]:
from langchain.output_parsers import PydanticOutputParser
actor_query="生成随机演员的电影作品表"

parser=PydanticOutputParser(pydantic_object=Actor)

In [12]:
misformatted="{''name':'Tom Hanks','film_names':['Forrest Gump']"
parser.parse(misformatted)

OutputParserException: Failed to parse Actor from completion {}. Got: 2 validation errors for Actor
name
  field required (type=value_error.missing)
film_names
  field required (type=value_error.missing)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [18]:
from langchain.output_parsers import OutputFixingParser

new_parser=OutputFixingParser.from_llm(parser=parser,llm=chat)

TypeError: Can't instantiate abstract class RunnableSerializable[OutputFixingParserRetryChainInput, str] with abstract method invoke

In [16]:
new_parser.parse(misformatted)


NameError: name 'new_parser' is not defined